In [1]:
import json
import tomllib
from pathlib import Path
from typing import Any

import httpx

### Read/Write Helper Functions
#### Write API Data Function

In [2]:
def write_api_data(
    endpoint_name: str, response: httpx.Response, data_dir: Path
) -> None:
    """Write an API response's JSON to file.

    Parameters
    ----------
    endpoint_name : str
        API endpoint name.
    response : httpx.Response
        Response returned from the GET request.
    data_dir : Path
        Path to the data directory for storing the output JSON file.
    """
    # location of the output JSON file.
    file_path = data_dir.joinpath(f"{endpoint_name}.json")
    with open(file_path, "w") as f:
        json.dump(response.json(), f)

#### Read API Data Function

In [3]:
def read_api_data(endpoint_name: str, data_dir: Path) -> dict[Any, Any]:
    """Read an API response's JSON from file.

    Parameters
    ----------
    endpoint_name : str
        API endpoint name.
    data_dir : Path
        Path to the data directory for storing the output JSON file.

    Returns
    -------
    dict[Any, Any]
        JSON returned as a dict.
    """
    # location of the input JSON file.
    file_path = data_dir.joinpath(f"{endpoint_name}.json")
    with open(file_path, "r") as f:
        data = json.load(f)
    return data

# Extract
### Paths

In [4]:
backend_dir = Path.cwd().parent
data_dir = backend_dir / "data"

### Read Secrets File Containing API Access Token and Account ID

In [5]:
with open(backend_dir / "secrets.toml", "rb") as f:
    secrets = tomllib.load(f)

### Build API Request Data

In [6]:
# Base URL for the tradier API.
base_url = "https://api.tradier.com/v1"
# Headers with access token.
headers = {
    "Authorization": f"Bearer {secrets['tradier']['access_token']}",
    "Accept": "application/json",
}
# Account information fragment.
account_fragment = f"/accounts/{secrets['tradier']['account_id']}"
# Fragments of account endpoints.
endpoint_fragments = {
    "user_profile": "/user/profile",
    "balances": f"{account_fragment}/balances",
    "positions": f"{account_fragment}/positions",
    "history": f"{account_fragment}/history",
    "gain_loss": f"{account_fragment}/gainloss",
    "orders": f"{account_fragment}/orders",
}

# Request data with url and headers for each request.
request_data: dict[str, dict[str, Any]] = {}
for endpoint_name, endpoint in endpoint_fragments.items():
    request_data[endpoint_name] = {
        "url": f"{base_url}{endpoint}",
        "headers": headers,
    }
# Add any parameters if necessary.
request_data["history"]["params"] = {"limit": 1000}
request_data["gain_loss"]["params"] = {"limit": 1000}

### Request Data from API

In [7]:
for endpoint_name, request in request_data.items():
    response = httpx.get(**request)
    write_api_data(endpoint_name, response, data_dir)